In [37]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex, hexcolor_str

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<!-- prettier-ignore-start -->

<article class="day-desc read-aloud"><h2>--- Day 11: Dumbo Octopus ---</h2><p>You enter a large cavern full of rare bioluminescent <a href="https://www.youtube.com/watch?v=eih-VSaS2g0" target="_blank">dumbo octopuses</a>! They seem to not like the Christmas lights on your submarine, so you turn them off for now.</p>
<p>There are 100 <span title="I know it's weird; I grew up saying 'octopi' too.">octopuses</span> arranged neatly in a 10 by 10 grid. Each octopus slowly gains <em>energy</em> over time and <em>flashes</em> brightly for a moment when its energy is full. Although your lights are off, maybe you could navigate through the cave without disturbing the octopuses if you could predict when the flashes of light will happen.</p>
<p>Each octopus has an <em>energy level</em> - your submarine can remotely measure the energy level of each octopus (your puzzle input). For example:</p>
<pre><code>5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526
</code></pre>
<p>The energy level of each octopus is a value between <code>0</code> and <code>9</code>. Here, the top-left octopus has an energy level of <code>5</code>, the bottom-right one has an energy level of <code>6</code>, and so on.</p>
<p>You can model the energy levels and flashes of light in <em>steps</em>. During a single step, the following occurs:</p>
<ul>
<li>First, the energy level of each octopus increases by <code>1</code>.</li>
<li>Then, any octopus with an energy level greater than <code>9</code> <em>flashes</em>. This increases the energy level of all adjacent octopuses by <code>1</code>, including octopuses that are diagonally adjacent. If this causes an octopus to have an energy level greater than <code>9</code>, it <em>also flashes</em>. This process continues as long as new octopuses keep having their energy level increased beyond <code>9</code>. (An octopus can only flash <em>at most once per step</em>.)</li>
<li>Finally, any octopus that flashed during this step has its energy level set to <code>0</code>, as it used all of its energy to flash.</li>
</ul>
<p>Adjacent flashes can cause an octopus to flash on a step even if it begins that step with very little energy. Consider the middle octopus with <code>1</code> energy in this situation:</p>
<pre><code>Before any steps:
11111
19991
19191
19991
11111

After step 1:
34543
4<em>000</em>4
5<em>000</em>5
4<em>000</em>4
34543

After step 2:
45654
51115
61116
51115
45654
</code></pre>

<p>An octopus is <em>highlighted</em> when it flashed during the given step.</p>
<p>Here is how the larger example above progresses:</p>
<pre><code>Before any steps:
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526

After step 1:
6594254334
3856965822
6375667284
7252447257
7468496589
5278635756
3287952832
7993992245
5957959665
6394862637

After step 2:
88<em>0</em>7476555
5<em>0</em>89<em>0</em>87<em>0</em>54
85978896<em>0</em>8
84857696<em>00</em>
87<em>00</em>9<em>0</em>88<em>00</em>
66<em>000</em>88989
68<em>0000</em>5943
<em>000000</em>7456
9<em>000000</em>876
87<em>0000</em>6848

After step 3:
<em>00</em>5<em>0</em>9<em>00</em>866
85<em>00</em>8<em>00</em>575
99<em>000000</em>39
97<em>000000</em>41
9935<em>0</em>8<em>00</em>63
77123<em>00000</em>
791125<em>000</em>9
221113<em>0000</em>
<em>0</em>421125<em>000</em>
<em>00</em>21119<em>000</em>

After step 4:
2263<em>0</em>31977
<em>0</em>923<em>0</em>31697
<em>00</em>3222115<em>0</em>
<em>00</em>41111163
<em>00</em>76191174
<em>00</em>53411122
<em>00</em>4236112<em>0</em>
5532241122
1532247211
113223<em>0</em>211

After step 5:
4484144<em>000</em>
2<em>0</em>44144<em>000</em>
2253333493
1152333274
11873<em>0</em>3285
1164633233
1153472231
6643352233
2643358322
2243341322

After step 6:
5595255111
3155255222
33644446<em>0</em>5
2263444496
2298414396
2275744344
2264583342
7754463344
3754469433
3354452433

After step 7:
67<em>0</em>7366222
4377366333
4475555827
34966557<em>0</em>9
35<em>00</em>6256<em>0</em>9
35<em>0</em>9955566
3486694453
8865585555
486558<em>0</em>644
4465574644

After step 8:
7818477333
5488477444
5697666949
46<em>0</em>876683<em>0</em>
473494673<em>0</em>
474<em>00</em>97688
69<em>0000</em>7564
<em>000000</em>9666
8<em>00000</em>4755
68<em>0000</em>7755

After step 9:
9<em>0</em>6<em>0000</em>644
78<em>00000</em>976
69<em>000000</em>8<em>0</em>
584<em>00000</em>82
5858<em>0000</em>93
69624<em>00000</em>
8<em>0</em>2125<em>000</em>9
222113<em>000</em>9
9111128<em>0</em>97
7911119976

After step 10:
<em>0</em>481112976
<em>00</em>31112<em>00</em>9
<em>00</em>411125<em>0</em>4
<em>00</em>811114<em>0</em>6
<em>00</em>991113<em>0</em>6
<em>00</em>93511233
<em>0</em>44236113<em>0</em>
553225235<em>0</em>
<em>0</em>53225<em>0</em>6<em>00</em>
<em>00</em>3224<em>0000</em>
</code></pre>

<p>After step 10, there have been a total of <code>204</code> flashes. Fast forwarding, here is the same configuration every 10 steps:</p>

<pre><code>After step 20:
3936556452
56865568<em>0</em>6
449655569<em>0</em>
444865558<em>0</em>
445686557<em>0</em>
568<em>00</em>86577
7<em>00000</em>9896
<em>0000000</em>344
6<em>000000</em>364
46<em>0000</em>9543

After step 30:
<em>0</em>643334118
4253334611
3374333458
2225333337
2229333338
2276733333
2754574565
5544458511
9444447111
7944446119

After step 40:
6211111981
<em>0</em>421111119
<em>00</em>42111115
<em>000</em>3111115
<em>000</em>3111116
<em>00</em>65611111
<em>0</em>532351111
3322234597
2222222976
2222222762

After step 50:
9655556447
48655568<em>0</em>5
448655569<em>0</em>
445865558<em>0</em>
457486557<em>0</em>
57<em>000</em>86566
6<em>00000</em>9887
8<em>000000</em>533
68<em>00000</em>633
568<em>0000</em>538

After step 60:
25333342<em>00</em>
274333464<em>0</em>
2264333458
2225333337
2225333338
2287833333
3854573455
1854458611
1175447111
1115446111

After step 70:
8211111164
<em>0</em>421111166
<em>00</em>42111114
<em>000</em>4211115
<em>0000</em>211116
<em>00</em>65611111
<em>0</em>532351111
7322235117
5722223475
4572222754

After step 80:
1755555697
59655556<em>0</em>9
448655568<em>0</em>
445865558<em>0</em>
457<em>0</em>86557<em>0</em>
57<em>000</em>86566
7<em>00000</em>8666
<em>0000000</em>99<em>0</em>
<em>0000000</em>8<em>00</em>
<em>0000000000</em>

After step 90:
7433333522
2643333522
2264333458
2226433337
2222433338
2287833333
2854573333
4854458333
3387779333
3333333333

After step 100:
<em>0</em>397666866
<em>0</em>749766918
<em>00</em>53976933
<em>000</em>4297822
<em>000</em>4229892
<em>00</em>53222877
<em>0</em>532222966
9322228966
7922286866
6789998766
</code></pre>
<p>After 100 steps, there have been a total of <code><em>1656</em></code> flashes.</p>
<p>Given the starting energy levels of the dumbo octopuses in your cavern, simulate 100 steps. <em>How many total flashes are there after 100 steps?</em></p>
</article>
<!-- prettier-ignore-end -->


In [38]:
from collections import deque
from collections.abc import Iterator
from copy import deepcopy
from tabulate import tabulate


example = """
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526
"""

special_case = """
11111
19991
19191
19991
11111
"""


class DumboOctopuses:
    ADJACENT = ((-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1))

    def __init__(self, s: str) -> None:
        self.energy_levels = [[int(el) for el in l] for l in s.strip().splitlines()]
        self.rows, self.cols = len(self.energy_levels), len(self.energy_levels[0])

    def step(self) -> int:
        rows, cols = self.rows, self.cols
        # First, the energy level of each octopus increases by 1.
        flashers = deque([])
        for r, c in product(range(rows), range(cols)):
            self.energy_levels[r][c] += 1
            if self.energy_levels[r][c] > 9:
                flashers.append(((r, c)))
        # Then, any octopus with an energy level greater than 9 flashes.
        # This increases the energy level of all adjacent octopuses by 1,
        # including octopuses that are diagonally adjacent.
        flashed = set()
        # This process continues as long as new octopuses keep having their
        # energy level increased beyond 9.
        while flashers:
            r, c = flashers.popleft()

            # (An octopus can only flash at most once per step.)
            if (r, c) in flashed:
                continue

            if self.energy_levels[r][c] > 9:
                flashed.add((r, c))
                # If this causes an octopus to have an energy level greater than 9,
                # it also flashes.
                for rr, cc in self.adjacent(r, c):
                    self.energy_levels[rr][cc] += 1
                    if self.energy_levels[rr][cc] > 9:
                        flashers.append(((rr, cc)))
        # Finally, any octopus that flashed during this step has its energy level set to 0,
        # as it used all of its energy to flash.
        for r, c in flashed:
            self.energy_levels[r][c] = 0

        return len(flashed)

    def adjacent(self, r: int, c: int) -> Iterator[tuple[int, int]]:
        yield from (
            (r + dr, c + dc)
            for dr, dc in self.ADJACENT
            if 0 <= r + dr < self.rows and 0 <= c + dc < self.cols
        )

    def __repr__(self) -> str:
        return "\n".join(
            "".join(
                f"{hexcolor_str('#FFFFE0' if e == 0 else '#708090' ,str(e))}" for e in r
            )
            for r in self.energy_levels
        )

    @classmethod
    def predict_and_count_flashes(cls, s: str, steps=100, do_print=False) -> int:
        dos = DumboOctopuses((s))
        if do_print:
            print("Before any steps:")
            print(dos)
            print()

        flashes = 0
        for step in range(1, steps + 1):
            flashes += dos.step()
            if do_print:
                print(f"After step {step}:")
                print(dos)
                print()

        return flashes

    @classmethod
    def first_step_all_flash(cls, s: str) -> int:
        dos = DumboOctopuses((s))

        step = 1
        while True:
            flashes = dos.step()
            if flashes == 100:
                return step

            step += 1


print(
    "FLashes:",
    DumboOctopuses.predict_and_count_flashes(special_case, steps=2, do_print=True),
)
assert DumboOctopuses.predict_and_count_flashes(example) == 1656

Before any steps:
11111
19991
19191
19991
11111

After step 1:
34543
40004
50005
40004
34543

After step 2:
45654
51115
61116
51115
45654

FLashes: 9


In [39]:
with open("../input/day11.txt") as f:
    puzzle = f.read()

print(f"Part I: {DumboOctopuses.predict_and_count_flashes(puzzle)}")

Part I: 1719


<link href="style.css" rel="stylesheet"></link>
<!-- prettier-ignore-start -->

<main>

<p>Your puzzle answer was <code>1719</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>It seems like the individual flashes aren't bright enough to navigate. However, you might have a better option: the flashes seem to be <em>synchronizing</em>!</p>
<p>In the example above, the first time all octopuses flash simultaneously is step <code><em>195</em></code>:</p>
<pre><code>After step 193:
5877777777
8877777777
7777777777
7777777777
7777777777
7777777777
7777777777
7777777777
7777777777
7777777777

After step 194:
6988888888
9988888888
8888888888
8888888888
8888888888
8888888888
8888888888
8888888888
8888888888
8888888888

After step 195:
<em>0000000000</em>
<em>0000000000</em>
<em>0000000000</em>
<em>0000000000</em>
<em>0000000000</em>
<em>0000000000</em>
<em>0000000000</em>
<em>0000000000</em>
<em>0000000000</em>
<em>0000000000</em>
</code></pre>

<p>If you can calculate the exact moments when the octopuses will all flash simultaneously, you should be able to navigate through the cavern. <em>What is the first step during which all octopuses flash?</em></p>
</article>

</main>


In [40]:
assert DumboOctopuses.first_step_all_flash(example) == 195

In [41]:
print(f"Part II: { DumboOctopuses.first_step_all_flash(puzzle)}")

Part II: 232


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>232</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
